In [1]:
import ujson as json
import calendar
import datetime, time
import time as t
import scipy
import numpy as np
import pandas as pd
start=t.time()
filename='Ogallala.json'
with open(filename, 'r') as f:
    points = json.load(f)
print len(points['features'])

21884


In [20]:
csv="jacknife_texas.csv"
jack_df=pd.read_csv(csv)
test_df=jack_df.head(5)
print(test_df)

      Well
0  1003903
1  1006909
2  1007701
3  1008102
4  1009601


In [31]:
start_date=1950
end_date=2010
interval=5
resolution=.01
time_tolerance=5
min_ratio=1
min_samples=20
iterations=int((end_date-start_date)/interval+1)

from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
from scipy.optimize import curve_fit
from scipy.interpolate import UnivariateSpline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
pd.options.display.max_rows=1000
pp = PdfPages('multipage.pdf')
firststart=t.time()

interpolation_df=pd.DataFrame()
combined_df = pd.DataFrame()
for well in points['features']:
    if 'TsTime' in well:
        if len(well['TsTime'])>min_samples and (np.array(well['TsTime']).max()-np.array(well['TsTime']).min())>pd.Timedelta(days=365*5).value/1000000000:
            if np.array(well['TsTime']).max()>calendar.timegm(datetime.datetime(end_date, 1, 1).timetuple()) and np.array(well['TsTime']).min()<calendar.timegm(datetime.datetime(start_date, 1, 1).timetuple()):
                name = str(well['properties']['HydroID'])
            elif jack_df['Well'].astype(str).str.contains(str(well['properties']['HydroID'])).any():
                name=str(well['properties']['HydroID'])+'Short'
                print("else")
            else:
                continue
            wells_df = pd.DataFrame(index=pd.to_datetime(well['TsTime'], unit='s', origin='unix'), data=well['TsValue'], columns=[name])
            wells_df.index.drop_duplicates(keep="first")
            wells_df = wells_df.resample('3M').mean()

            combined_df = pd.concat([combined_df, wells_df], join="outer", axis=1)
            combined_df.drop_duplicates(inplace=True)
#combined_df is a pandas dataframe containing the ts depth to water table values for each well in the aquifer with the min number of points
# The columns of combined_df are the HydroIDs of the wells. If the well timeseries data spans the period of interpolation, the column title
# is the HydroID. If the timeseries does not span the time period, then the column title is the HydroID + "Short"
# The rows of the database are the timeseries values for 3 month intervals from start_date to end_date
print("Done with well loop")
corrs_df = combined_df.interpolate(method='pchip', limit_area='inside', limit=8)
combined_df.interpolate(method='pchip', inplace=True, limit_area='inside')

#This code resamples the data so we are working on March data since that is what Utah records
combined_df=combined_df.resample('Q').nearest()
decemb=combined_df.index.get_loc(str(start_date)+'-12-31 00:00:00',method='nearest') % 4
combined_df=combined_df.iloc[decemb::4]

combined_df['linear']=np.linspace(0,1,len(combined_df))
interpolation_df = combined_df.drop(combined_df.filter(regex='Short').columns, axis=1)
corr_df=corrs_df.corr(min_periods=12)#12
corr_df=corr_df-np.identity(len(corr_df))
length=len(corr_df)-1
for row in corr_df.index:
    if 'Short' in str(row):
        corr_df=corr_df.drop(row)
print t.time()-firststart
print(combined_df.columns)

mlrs=[]
acts=[]
ws=[]
for i in range(len(jack_df)):
    jacknife=jack_df['Well'][i]
    print(jacknife)

    well_list=[]
    for i in range(length):#length
        reflist=np.array(corr_df.nlargest(5,corr_df.columns[i]).index)
        welli = corr_df.columns[i]
        well_list.append(welli)
        if str(jacknife) in str(welli):
            mydf = combined_df[[welli]].copy()
            print "Well ",welli
            corr_values=corr_df.nlargest(5, corr_df.columns[i]).values[:,i]
            print(corr_values)
            reflist=np.append(reflist,'linear')
            delete_list=[]
            for j in range(len(corr_values)):
                if corr_values[j]<=0.5:
                    delete_list.append(j)
            if len(delete_list)>0:
                reflist=np.delete(reflist,delete_list)
            if len(reflist)>=1:
                for ref in reflist:
                    mydf=pd.concat([mydf,combined_df[ref]], axis=1)
                norm_exdf=mydf.dropna(how="any", subset=reflist)
                normz_exdf=norm_exdf.copy()
                smaller_exdf=normz_exdf.copy()
                smaller_exdf=smaller_exdf.dropna(how="any")
                for column in norm_exdf.columns[0:]:
                    normz_exdf[column]=(norm_exdf[column]-smaller_exdf[column].min())/(smaller_exdf[column].max()-smaller_exdf[column].min())
                exdf = normz_exdf.copy()
                mymin=max(exdf[welli].first_valid_index(),exdf[reflist[0]].first_valid_index())
                mymax="1995-01-01 00:00:00"
                exdf=exdf[mymin:mymax]
                y=exdf[welli]
                x=exdf.as_matrix(columns=reflist)
                n,m=x.shape
                x0=np.ones((n,1))
                x=np.hstack((x,x0))
                model=sm.OLS(y,x)
                model_fit = model.fit_regularized(alpha=0.005)
                b=np.array(model_fit.params)

                A=normz_exdf.as_matrix(columns=reflist)
                n,m=A.shape
                A0=np.ones((n,1))
                A=np.hstack((A,A0))
                norm_exdf['prediction']=np.dot(A,b)
                norm_exdf['prediction']=norm_exdf['prediction']*(norm_exdf[welli].max()-norm_exdf[welli].min())+norm_exdf[welli].min()
                #toggle this next line of code
    #             norm_exdf.loc[corrs_df[welli].notnull(),'prediction']=corrs_df[welli]
                newname=str(welli).replace('Short','')
                interpolation_df = pd.concat([interpolation_df, norm_exdf['prediction']], join="outer", axis=1)
                interpolation_df=interpolation_df.rename(columns={"prediction":newname})
                plt.plot(combined_df[welli], 'r', label='Original Data')
                plt.plot(norm_exdf['prediction'], 'r--', label='MLR Prediction')
                plt.plot(combined_df[welli][mymin:mymax], 'g', label='Training Data')
                combined_df[welli+'naive']=combined_df[welli]
                combined_df[welli+'naive'][mymax:]=combined_df[welli+'naive'][combined_df.index.get_loc(mymax,method='nearest')]
                combined_df[welli+'prediction']=norm_exdf['prediction']
                if "Short" in welli:
                    well_title="Well "+welli[:-5]
                else:
                    well_title="Well "+welli
                plt.title(well_title)
                plt.legend()
                plt.xlabel('Date')
                plt.ylabel('Depth to GW (ft)')
                pp.savefig()
                plt.close()
                test_index=combined_df.index.get_loc('2010-01-01 00:00:00',method='nearest')
                mlr_v=combined_df[welli+'prediction'][test_index]
                act_v=(combined_df[welli][test_index])
                mlrs.append(mlr_v)
                acts.append(act_v)
                ws.append(welli)
pp.close()
plt.show()
print "finished"
print t.time()-firststart
jack_df['mlr']=mlrs
jack_df['actual']=acts
print(jack_df)

else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else
else


/home/student/tethys/miniconda/envs/tethys/lib/python2.7/site-packages/ipykernel_launcher.py:101: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/home/student/tethys/miniconda/envs/tethys/lib/python2.7/site-packages/ipykernel_launcher.py:109: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/home/student/tethys/miniconda/envs/tethys/lib/python2.7/site-packages/ipykernel_launcher.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/student/tethys/miniconda/envs/tethys/lib/python2.7/site-packages/ipykernel_launcher.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

1006909
Well  1006909Short
[0.99303587 0.99292178 0.99060351 0.98991531 0.98898773]
1007701
Well  1007701
[0.98195462 0.95795949 0.93636829 0.93545516 0.92790273]
1008102
Well  1008102Short
[0.9713278  0.97027508 0.96941163 0.96709772 0.96623152]
1009601
Well  1009601Short
[0.84879638 0.76484204 0.70759313 0.66950563 0.63178705]
1011404
Well  1011404Short
[0.99300565 0.99114907 0.99099509 0.98693809 0.98639729]
1012203
Well  1012203Short
[0.78367144 0.69289858 0.59643597 0.56446908 0.52969984]
1014205
Well  1014205Short
[0.97830029 0.93467981 0.90296274 0.90059247 0.88682754]
1014513
Well  1014513Short
[0.99674161 0.99363168 0.99044829 0.98820253 0.98731916]
1014704
Well  1014704
[0.99704253 0.99638823 0.99630537 0.99626507 0.99375738]
1014901
Well  1014901
[0.98412649 0.98213409 0.98104295 0.97656742 0.97472582]
1017501
Well  1017501Short
[0.9742037  0.9679816  0.94218338 0.93894448 0.93052682]
1018701
Well  1018701Short
[0.91822265 0.91725637 0.91506849 0.87833959 0.8741723 ]
1018901

2309601
Well  2309601Short
[0.96974551 0.96963003 0.96425279 0.95673782 0.95588573]
2311401
Well  2311401
[0.99814613 0.99694769 0.99638823 0.99615448 0.99587111]
2311601
Well  2311601
[0.99678411 0.98924965 0.98476181 0.98404929 0.98305295]
2311702
Well  2311702Short
[0.99501213 0.99388102 0.99311451 0.992221   0.99137303]
2312301
Well  2312301
[0.99311193 0.98834207 0.9846518  0.98288218 0.98137824]
2312401
Well  2312401Short
[0.99716673 0.99361524 0.98923117 0.98621738 0.98544813]
2312402
Well  2312402Short
[0.99240568 0.98784788 0.98765975 0.98308719 0.97898219]
2312801
Well  2312801
[0.99551122 0.99535727 0.99524691 0.99128292 0.99061706]
2312803
Well  2312803Short
[0.99908516 0.99374233 0.99159313 0.99111807 0.99005126]
2312902
Well  2312902
[0.99675472 0.99521602 0.99311078 0.9919896  0.9914131 ]
2315204
Well  2315204Short
[0.98772518 0.98506554 0.98496101 0.98328476 0.98224116]
2315501
Well  2315501Short
[0.98461966 0.98265655 0.98208204 0.98180732 0.9790018 ]
2316101
Well  231

2421905
Well  2421905Short
[0.98597894 0.97948754 0.97764296 0.97655737 0.96338397]
2422201
Well  2422201Short
[0.87801098 0.84228716 0.68428761 0.68389674 0.61285165]
2422202
Well  2422202Short
[0.81738222 0.80058072 0.78226445 0.77001076 0.76126203]
2424402
Well  2424402
[0.98785026 0.98729429 0.98476181 0.98447303 0.98412649]
2424903
Well  2424903Short
[0.97743056 0.97560227 0.97390862 0.97282742 0.97139686]
2427201
Well  2427201Short
[0.96224235 0.95065234 0.94711464 0.92383939 0.92327075]
2429901
Well  2429901
[0.97249882 0.96995812 0.96988494 0.96933291 0.96646852]
2431601
Well  2431601
[0.83547413 0.79275315 0.77340784 0.77232519 0.73524866]
2432305
Well  2432305
[0.90241406 0.89636218 0.86628978 0.84366825 0.84228273]
2437204
Well  2437204Short
[0.95816791 0.95808427 0.94287422 0.93713374 0.93317848]
2439901
Well  2439901Short
[0.88469838 0.72002538 0.71105495 0.69462075 0.68592953]
2439904
Well  2439904Short
[0.91386629 0.8870882  0.87763556 0.74413618 0.67063755]
2440207
2440

338803
Well  338803Short
[0.98733154 0.98512206 0.98376911 0.98241493 0.97991776]
338901
Well  338901Short
[0.99333718 0.99172316 0.98955348 0.98906197 0.98873716]
341502
Well  341502Short
[0.99330793 0.98308928 0.98060671 0.97984879 0.97916677]
341602
Well  341602Short
[0.99402694 0.99368161 0.9934816  0.99304311 0.99198622]
341801
Well  341801Short
[0.99680277 0.98841318 0.98629481 0.98493559 0.98337248]
344501
Well  344501Short
[0.9939313  0.99213674 0.99022857 0.98930968 0.98904114]
345801
Well  345801Short
[0.9944359  0.99400638 0.99340965 0.99292366 0.99232423]
347902
Well  347902Short
[0.99020937 0.98656908 0.98540528 0.98528952 0.98411383]
348404
Well  348404Short
[0.94910453 0.92653846 0.92245058 0.91908783 0.91413435]
348802
Well  348802Short
[0.99272414 0.98958736 0.98900762 0.98664866 0.98545758]
348906
Well  348906Short
[0.99432484 0.99321346 0.99041274 0.99013689 0.98789044]
349601
Well  349601Short
[0.9884201  0.98559963 0.97966191 0.97243466 0.96963818]
352301
Well  352

661101
Well  661101Short
[0.23514615 0.22332681 0.21652368 0.20359533 0.20054794]
661301
Well  661301Short
[0.83554519 0.79177142 0.76400363 0.72391009 0.68359264]
662101
Well  662101Short
[0.96880377 0.96095759 0.95412288 0.9511709  0.94863266]
662501
Well  662501Short
[0.74259877 0.70511766 0.70183736 0.68649235 0.68308682]
663801
Well  663801Short
[0.84629365 0.84388686 0.83481021 0.83105945 0.83066462]
664501
Well  664501Short
[0.83741595 0.83493562 0.82328755 0.80268799 0.79507032]
664701
Well  664701Short
[0.89822662 0.86223938 0.8520167  0.83739507 0.83101947]
664702
Well  664702Short
[0.84005899 0.80306016 0.78474153 0.78342963 0.77918805]
701203
Well  701203Short
[0.97190022 0.96003259 0.93413774 0.93385362 0.92686753]
716302
Well  716302Short
[0.96549472 0.9616142  0.95431724 0.95305579 0.9530257 ]
716602
Well  716602Short
[0.96335083 0.8666616  0.85356702 0.85208016 0.84238187]
754404
Well  754404Short
[0.9658738  0.96485373 0.96180185 0.95769907 0.95093171]
755601
Well  755

ValueError: Length of values does not match length of index

In [34]:
print(mlrs)
print(acts)
print(ws)

[-260.0385065996397, -172.55755341071313, -120.11653596440325, -152.79456055270947, -58.422986479282734, -206.7731388881925, -75.66948968697793, -103.18103081016008, -99.15728545952437, -170.2443172127592, -112.885355007409, -264.9617361379702, -254.98279585681038, -291.78000382705466, -302.15920040257566, -191.84744072826618, -190.811051138418, -302.8355796861897, -351.67535943955295, -188.7687171894209, -288.47615788088495, -304.6994738615182, -324.6094834362234, -261.0795289463051, -270.512187490766, -190.6254181999024, -203.6217686239949, -172.96078224947578, -239.95349160919778, -115.62505925262928, -99.83310367506482, -90.10010091045547, -115.56498295089514, -115.65003729173807, -88.18191524396478, -77.97331178546074, -115.79220431361986, -272.0747034407842, -182.00900929911967, -282.7912783576376, -119.17079325528485, -129.80018064831197, -177.86617431405983, -170.26066479126376, -7.546080426817352, -120.64929711419458, -70.0043171161121, -164.16401780560605, -191.72739076660434

In [ ]:
newinterpolation_df=interpolation_df[str(start_date):str(end_date)].resample('5Y',closed='left').nearest()
corrs_df=corrs_df[str(start_date):str(end_date)].resample('5Y',closed='left').nearest()
corrs_df.rename(columns=lambda x: x.replace('Short',''),inplace=True)
lons=[]
lats=[]
values=[]
elevations=[]
ids=[]
mylon=[]
mylat=[]
myelevs=[]
myids=[]
for wellid in newinterpolation_df.columns:
    for well in points['features']:
        if wellid==str(well['properties']['HydroID']):
            mylon.append(well['geometry']['coordinates'][0])
            mylat.append(well['geometry']['coordinates'][1])
            myelevs.append(well['properties']['LandElev'])
            myids.append(wellid)
            newinterpolation_df.loc[corrs_df[wellid].notnull(),wellid]=corrs_df[wellid]
            break
for i in range(iterations):
    myvalue=np.array(newinterpolation_df.iloc[i].tolist())
    myelev=np.add(np.array(myelevs),myvalue)
    lons.append(mylon)
    lats.append(mylat)
    values.append(myvalue)
    elevations.append(myelev)  
    ids.append(myids)
    print len(mylon)
lons = np.array(lons)
lats = np.array(lats)
ids=np.array(ids)
values = np.array(values)
elevations = np.array(elevations)
print "done"

In [ ]:
import matplotlib.pyplot as plt

nlags=6
#Variogram
def distance(lon1,lat1,lon2,lat2):
    d2=(lon2-lon1)**2+(lat2-lat1)**2
    d=d2**(.5)
    return d
def deviation(value1,value2):
    diff=(value1-value2)**2
    return diff

all_distances=[]
all_deviations=[]
for step in range(0,len(lons)):
    distances=[]
    deviations=[]
    lon=lons[step]
    lat=lats[step]
    value=values[step]
    elevation=elevations[step]
    for x in range(0,len(lon)):
        for y in range(0,len(lon)):
            if x==y:
                continue
            myx=distance(lon[x],lat[x],lon[y],lat[y])
            myy=deviation(value[x],value[y])
            distances.append(myx)
            deviations.append(myy)
    distances=np.array(distances)
    deviations=np.array(deviations)
    max=np.max(distances)
    min=np.min(distances)
    size=((max-min)/4)/nlags

    xbins=[]
    ybins=[]
    meanx=[]
    meany=[]
    
    for c in range(0,(nlags*4)):
        xbin1=[]
        ybin1=[]
        for d in range(0,len(distances)):
            if distances[d]>=(min+size*c) and distances[d]<(min+size*(c+1)):
                xbin1.append(distances[d])
                ybin1.append(deviations[d])
        xbin1=np.array(xbin1)
        ybin1=np.array(ybin1)
        meanx1=np.average(xbin1)
        meany1=np.average(ybin1)
        xbins.append(xbin1)
        ybins.append(ybin1)
        meanx.append(meanx1)
        meany.append(meany1/2)
    
    plt.figure(figsize=(12, 6))
#     plt.scatter(distances, deviations, label='Data')
    plt.scatter(meanx,meany,label='Average')
    plt.show() 
    all_distances.append(distances)
    all_deviations.append(deviations)
all_distances=np.array(all_distances)
all_deviations=np.array(all_distances)
            




In [ ]:
lonmin=360.0
latmin=90.0
lonmax=-360.0
latmax=-90.0
for i in points['features']:
    if i['geometry']['coordinates'][0]<lonmin:
        lonmin=i['geometry']['coordinates'][0]
    if i['geometry']['coordinates'][0]>lonmax:
        lonmax=i['geometry']['coordinates'][0]
    if i['geometry']['coordinates'][1]<latmin:
        latmin=i['geometry']['coordinates'][1]
    if i['geometry']['coordinates'][1]>latmax:
        latmax=i['geometry']['coordinates'][1]
lonmin=round(lonmin-.05,1)
latmin=round(latmin-.05,1)
lonmax=round(lonmax+.05,1)
latmax=round(latmax+.05,1)
print lonmin,lonmax,latmin,latmax
print len(points['features'])

In [ ]:
import numpy as np
latgrid = np.mgrid[latmin:latmax:resolution]
longrid = np.mgrid[lonmin:lonmax:resolution]
latrange=len(latgrid)
lonrange=len(longrid)
print latrange, lonrange


In [ ]:
import pygslib
outx=np.repeat(longrid,latrange)
outy=np.tile(latgrid,lonrange)
grids=[]
for i in range(0,iterations):
    params={
        'x':lons[i],
        'y':lats[i],
        'vr':values[i],
        'nx':lonrange,
        'ny':latrange,
        'nz':1,
        'xmn':lonmin,
        'ymn':latmin,
        'zmn':0,
        'xsiz':resolution,
        'ysiz':resolution,
        'zsiz':1,
        'nxdis':1,
        'nydis':1,
        'nzdis':1,
        'outx':outx,
        'outy':outy,
        'radius':2,
        'radius1':2,
        'radius2':2,
        'ndmax':15,
        'ndmin':5,
        'noct':0,
        'ktype':1,
        'idbg':0,
        'c0':0,
        'it':1,
        'cc':500,
        'aa':.2,
        'aa1':.2,
        'aa2':.2
    }
    estimate= pygslib.gslib.kt3d(params)
    print i
    array=estimate[0]['outidpower']#outest
    array2=estimate[0]['outidpower']
    grid=np.reshape(array,(lonrange,latrange))
    grid2=np.reshape(array2,(lonrange,latrange))
    x=np.isnan(grid)
    grid[x]=grid2[x]
    grids.append(grid)
grids=np.array(grids)

In [ ]:
with open('UtahMajorAquifers.json','r') as f:
    allwells = ''
    wells = f.readlines()
    for i in range(0, len(wells)):
        allwells += wells[i]
utah=json.loads(allwells)


AquiferShape= {
    'type':'FeatureCollection',
    'features':[]
}
print utah['features'][0].viewkeys()
for i in utah['features']:
    if i['properties']['Full_Name']=="Cedar Valley":#Full_Name #AQ_NAME TRINITY
        AquiferShape['features'].append(i)

with open('shapefile.json','w') as outfile:
    json.dump(AquiferShape, outfile)
    

In [ ]:
from __future__ import division
import netCDF4
from netCDF4 import Dataset
import numpy as np
import os
import time as t
from scipy import interpolate
from sklearn.utils import resample
import pykrige.kriging_tools as kt
from pykrige.ok import OrdinaryKriging
import numpy as np
import matplotlib.pyplot as plt
import cython
hydros= np.array(ids[0])
volunit="Acre-ft"

interpolation_type="Kriging"
# f=idw(spots,values)
# print f([-96.8,32.7],power=1.0)
# f([-96.0,30.7],power=2,n_neighbors=len(values))

latlen=len(latgrid)
lonlen=len(longrid)
filename='testinterpolate2.nc'
h=netCDF4.Dataset(filename,'w',format="NETCDF4")
h.interpolation="Kriging Multi Linear"
h.start_date=start_date
h.end_date=end_date
h.interval=interval
h.resolution=resolution
h.min_samples=min_samples
h.min_ratio=min_ratio
h.time_tolerance=time_tolerance
h.default=0
h.units='English'


time = h.createDimension("time", 0)
lat = h.createDimension("lat", latlen)
lon = h.createDimension("lon", lonlen)
hydroid=h.createDimension("hydroid", len(ids[0]))
latitude=h.createVariable("lat",np.float64,("lat"))
longitude=h.createVariable("lon",np.float64,("lon"))
time=h.createVariable("time",np.float64,("time"), fill_value="NaN")
hydroids=h.createVariable("hydroid",str,("hydroid"), fill_value="NaN")
hydroids.axis="H"

tsvalue=h.createVariable("tsvalue",np.float64,('time','hydroid'),fill_value=-9999)
tsvalue.units="ft"
tsvalue.coordinates="time hydroid"

depth=h.createVariable("depth",np.float64,('time','lon','lat'),fill_value=-999)
depth.long_name="Depth to Groundwater"
depth.units="ft"
depth.grid_mapping="WGS84"
depth.cell_measures="area: area"
depth.coordinates="time lat lon"

elevation=h.createVariable("elevation",np.float64,('time','lon','lat'),fill_value=-999)
elevation.long_name="Elevation of Groundwater"
elevation.units="ft"
elevation.grid_mapping="WGS84"
elevation.cell_measures="area: area"
elevation.coordinates="time lat lon"

drawdown=h.createVariable("drawdown",np.float64,('time','lon','lat'),fill_value=-999)
drawdown.long_name="Well Drawdown"
drawdown.units="ft"
drawdown.grid_mapping="WGS84"
drawdown.cell_measures="area: area"
drawdown.coordinates="time lat lon"

volume = h.createVariable("volume", np.float64, ('time', 'lon', 'lat'), fill_value=-9999)
volume.long_name = "Change in aquifer storage volume since " + str(start_date)
volume.units = volunit
volume.grid_mapping = "WGS84"
volume.cell_measures = "area: area"
volume.coordinates = "time lat lon"

latitude.long_name="Latitude"
latitude.units="degrees_north"
latitude.axis="Y"
longitude.long_name="Longitude"
longitude.units="degrees_east"
longitude.axis="X"
time.axis="T"
time.units='days since 0001-01-01 00:00:00 UTC'
latitude[:]=latgrid[:]
longitude[:]=longrid[:]
hydroids[:]=ids[0,:]
year=start_date
timearray=[]#[datetime.datetime(2000,1,1).toordinal()-1,datetime.datetime(2002,1,1).toordinal()-1]
            
for i in range(0,iterations):
    print i
    timearray.append(datetime.datetime(year, 1, 1).toordinal() - 1)
    year += interval
    time[i] = timearray[i]
    tsvalue[i,:]=values[i,:]
    for x in range(0,lonrange):
            depth[i,x,:]=grids[i,x,:]
            elevation[i,x,:]=grids[i,x,:]
            if i==0:
                drawdown[i,x,:]=0
            else:
                drawdown[i,x,:]=depth[i,x,:]-depth[0,x,:]

h.close()
os.system("./aquifersubset.sh %s" % (filename))
# serverpath="/home/student/tds/apache-tomcat-8.5.30/content/thredds/public/testdata/groundwater"
# destination= os.path.join(serverpath, filename)
# os.rename(filename, destination)
end=t.time()
print(end-start)



In [ ]:
# os.system("./aquifersubset.sh %s" % (filename))

In [ ]:
import urllib
import copy
from xml.etree import cElementTree as ET
# This function returns the first and last available time from a url of a getcapabilities page located on a Thredds Server
def getTimeBounds(url):
    f=urllib.urlopen(url)
    tree=ET.parse(f)
    root=tree.getroot()
    #These lines of code find the time dimension information for the netcdf on the Thredds server
    dim=root.findall('.//{http://www.opengis.net/wms}Dimension')
    dim=dim[0].text
    times=dim.split(',')
    times.pop(0)
    timemin=times[0]
    timemax=times[-1]
    #timemin and timemax are the first and last available times on the specified url
    return timemin,timemax
#This function returns a pandas dataframe of the timeseries values of a specific layer 
#at a specific latitude and longitude from a file on a Thredds server
#server: the url of the netcdf desired netcdf file on the Thredds server to read
#layer: the name of the layer to extract timeseries information from for the netcdf file
#lat: the latitude of the point at which to extract the timeseries
#lon: the longitude of the point at which to extract the timeseries
#returns df: a pandas dataframe of the timeseries at lat and lon for the layer in the server netcdf file
def getThreddsValue(server,layer,lat,lon):
    #calls the getTimeBounds function to get the first and last available times for the netcdf file on the server
    timemin,timemax=getTimeBounds(server+"?service=WMS&version=1.3.0&request=GetCapabilities")
    #These lines properly format a url request for the timeseries of a speific layer from a netcdf on 
    #a Thredds server
    server=server+"?service=WMS&version=1.3.0&request=GetFeatureInfo&CRS=CRS:84&QUERY_LAYERS="+layer
    server=server+"&X=0&Y=0&I=0&J=0&BBOX="+str(lon)+','+str(lat)+','+str(lon+.001)+','+str(lat+.001)
    server=server+"&WIDTH=1&Height=1&INFO_FORMAT=text/xml"
    server=server+'&TIME='+timemin+'/'+timemax
    f = urllib.urlopen(server)
    tree=ET.parse(f)
    root=tree.getroot()
    features=root.findall('FeatureInfo')
    times=[]
    values=[]
    for child in features:
        time=datetime.datetime.strptime(child[0].text,"%Y-%m-%dT%H:%M:%S.%fZ")
        times.append(time)
        values.append(child[1].text)
    
    df=pd.DataFrame(index=times, columns=[layer],data=values)
    df[layer]=df[layer].replace('none',np.nan).astype(float)
    return df

#function to convert a datetime object to milliseconds since epoch
def datetime_to_float(d):
    epoch = datetime.datetime.utcfromtimestamp(0)
    total_seconds =  (d - epoch).total_seconds()
    # total_seconds will be in decimals (millisecond precision)
    return total_seconds


server="http://localhost:8080/thredds/wms/testAll/groundwater/testinterpolate2.nc"
layer="depth"
#calls a function to return a df of the timeseries of the pdsi at 37.7,-113
df1=getThreddsValue(server,layer,jacklat,jacklon)
print(df1)
print(jacklat,jacklon)